In [ ]:
import sys

sys.path.append("..")

In [ ]:
import numpy as np
import pandas as pd
import torch
import os

from experiments import Experiment

In [ ]:
exps = {
    "CA Housing": "../experiments/california_housing_reg.pkl",
    "CIFAR10": "../experiments/cifar10_cnn.pkl",
    "MNIST": "../experiments/mnist_fc.pkl",
}
split_layers = {
    "CIFAR10": "relu3",
    "MNIST": "relu0",
}

In [ ]:
df = list()
for name, path in exps.items():
    row = dict()
    e, row["Dataset Name"] = Experiment.load(path), name
    row["Experiment"] = e
    e.get_model(split_layer=None, root="..")
    # row["Full Model"] = e.get_model(split_layer=None, root="..")

    train_dataset = e.train_dataset
    test_dataset = e.test_dataset

    # print(e.__dict__)
    if hasattr(e, "train_args"):
        e.train_kwargs = e.train_args
    row["Learning Rate"] = e.train_kwargs["lr"]
    row["Batch Size"] = e.train_kwargs["batch_size"] if "batch_size" in e.train_kwargs else 64
    row["Epochs"] = e.train_kwargs["epochs"] if "epochs" in e.train_kwargs else 10

    row["Dataset Size"] = len(train_dataset) + len(test_dataset)

    if train_dataset.task == "classification":
        row["Task"] = "Classification"
        row["# Classes"] = len(np.unique(train_dataset.y))
        # row["Train Accuracy / $R^2$"] = train_dataset.get_accuracy(e.model)
        row["Test Accuracy"] = test_dataset.get_accuracy(e.model)
        row["Test AUC"] = test_dataset.get_auc(e.model)
        # row ["Train AUC"] = train_dataset.get_auc(e.model)
    elif train_dataset.task == "regression":
        row["Task"] = "Regression"
        # row["Train Accuracy / $R^2$"] = train_dataset.get_rsquared(e.model)
        row["Test $R^2$"] = test_dataset.get_rsquared(e.model)
        # row["Train AUC / MSE"] = train_dataset.get_mse(e.model)
        row["Test MSE"] = test_dataset.get_mse(e.model)

    if np.prod(e.model.input_shape) > 10:
        embedder, model = e.get_model(split_layer=split_layers[name], root="..")
    else:
        embedder, model = None, e.get_model(root="..")
    
    
    # row["Cut Model"] = model
    layers = list(model.layers.values())
    widths = [layer.in_features for layer in layers if isinstance(layer, torch.nn.Linear)] + [layers[-1].out_features]
    row["Hidden Layer Sizes"] = ', '.join(str(w) for w in widths[1:-1])
    row["Input Dimension"] = widths[0]

    df.append(row)
df = pd.DataFrame(df)
df

In [ ]:
nice_df = df.set_index("Dataset Name").T.fillna("NA").loc[["Dataset Size", "Task", "# Classes", "Batch Size", "Epochs", "Learning Rate", "Input Dimension", "Hidden Layer Sizes", "Test Accuracy", "Test AUC", "Test $R^2$", "Test MSE"]]
nice_df

In [ ]:
os.makedirs("../figures/", exist_ok=True)
with open("../figures/network_summary.tex", "w") as f:
    # f.write(df.set_index("Dataset Name")[["Dataset Size", "Input Dimension", "Hidden Layer Sizes", "Accuracy / $R^2$", "AUC / MSE"]].to_latex().replace("#", "\\#").replace("%", "\\%"))
    f.write(nice_df.to_latex(float_format="%.2f").replace("#", "\\#").replace("%", "\\%"))